In [1]:
import pandas as pd

splits = {'train': 'labeled_final/train-00000-of-00001.parquet', 'test': 'labeled_final/test-00000-of-00001.parquet', 'validation': 'labeled_final/validation-00000-of-00001.parquet'}
df = pd.read_parquet("hf://datasets/google-research-datasets/paws/" + splits["train"])

/Users/minhajul/miniconda3/envs/ide2/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df

,id,sentence1,sentence2,label
0,1,"In Paris , in October 1560 , he secretly met t...","In October 1560 , he secretly met with the Eng...",0
1,2,The NBA season of 1975 -- 76 was the 30th seas...,The 1975 -- 76 season of the National Basketba...,1
2,3,"There are also specific discussions , public p...","There are also public discussions , profile sp...",0
3,4,When comparable rates of flow can be maintaine...,The results are high when comparable flow rate...,1
4,5,It is the seat of Zerendi District in Akmola R...,It is the seat of the district of Zerendi in A...,1
...,...,...,...,...
49396,49397,"`` Our school is of spiritual and spiritual , ...",`` Our School is of the Temporal and the Spiri...,0
49397,49398,She was in Cork on June 24 and arrived on 8 Ju...,"She was at Cork on 24 June , and arrived in th...",1
49398,49399,Cornelia Stuyvesant Vanderbilt ( George and Ed...,John John F. A. Cecil ( the only child of Geor...,0
49399,49400,The third season was premiered on 7 June 2010 ...,"The fourth season was premiered on June 7 , 20...",0


In [3]:
df.label.value_counts()

label
0    27572
1    21829
Name: count, dtype: int64

In [4]:
# Take a subset with 50/50 distribution of labels 0 and 1
label_0 = df[df['label'] == 0].sample(n=5000, random_state=42)
label_1 = df[df['label'] == 1].sample(n=5000, random_state=42)

# Combine the subsets
balanced_df = pd.concat([label_0, label_1])

# Shuffle the combined dataframe
balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)

print(f"Shape of balanced subset: {balanced_df.shape}")
print("\nLabel distribution in balanced subset:")
print(balanced_df['label'].value_counts(normalize=True))

# Display the first few rows of the balanced subset
balanced_df.head()


Shape of balanced subset: (10000, 4)

Label distribution in balanced subset:
label
1    0.5
0    0.5
Name: proportion, dtype: float64


,id,sentence1,sentence2,label
0,17344,Methods of the cubic geometry provide the foll...,Methods of cubic geometry provide the followin...,1
1,157,The system required the user to encode the qua...,The system required the user to encode the def...,0
2,27825,They also meet the twins Timmy and Tommy Resto...,They soon encounter twins Timmy and Tommy Rest...,0
3,6712,The music was composed by V. Dakshinamoorthy a...,The music was written by V. Dakshinamoorthy an...,0
4,29787,was the brother of Charles Fenton Mercer and f...,"Mercer was the brother of George Mercer , John...",0


In [6]:
balanced_df.to_parquet("../storage/auto_eval_research/paws/balanced_df.parquet")